In [3]:
import gc
import pprint

class Graph:
    def __init__(self, name):
        self.name = name
        self.next = None
        
    def set_next(self, next):
        print(f'link nodes {self}.next={next}')
        self.next = next
        
    def __repr__(self):
        return f"{self.__class__.__name__}({self.name})"
    
one = Graph('one')
two = Graph('two')
three = Graph('three')
one.set_next(two)
two.set_next(three)
three.set_next(one)

for r in gc.get_referents(three):
    pprint.pprint(r)

link nodes Graph(one).next=Graph(two)
link nodes Graph(two).next=Graph(three)
link nodes Graph(three).next=Graph(one)
{'name': 'three', 'next': Graph(one)}
<class '__main__.Graph'>


In [4]:
gc.get_referents(three)  # get_referents() 函数可以根据输入的参数显示对象的 引用目标

[{'name': 'three', 'next': Graph(one)}, __main__.Graph]

In [5]:
import queue

seen = set()
to_process = queue.Queue()

to_process.put(([], three))

while not to_process.empty():
    chain, next = to_process.get()
    chain = chain[:]
    chain.append(next)
    print('examining', repr(next))
    seen.add(id(next))
    for r in gc.get_referents(next):
        if isinstance(r, str) or isinstance(r, type):
            pass
        elif id(r) in seen:
            print('found a circle to ', r)
            for i, link in enumerate(chain):
                print(f" {i}:", end=' ')
                pprint.pprint(link)
        else:
            to_process.put((chain, r))

examining Graph(three)
examining {'name': 'three', 'next': Graph(one)}
examining Graph(one)
examining {'name': 'one', 'next': Graph(two)}
examining Graph(two)
examining {'name': 'two', 'next': Graph(three)}
found a circle to  Graph(three)
 0: Graph(three)
 1: {'name': 'three', 'next': Graph(one)}
 2: Graph(one)
 3: {'name': 'one', 'next': Graph(two)}
 4: Graph(two)
 5: {'name': 'two', 'next': Graph(three)}


In [6]:
# 从模块的命名空间中删除这些节点的引用
one = two = three = None

for i in range(2):
    print(f'\nCollecting {i}')
    n = gc.collect()
    print('unreachable objs: ',n)
    print('Remaining Garbage:')
    pprint.pprint(gc.garbage)


unreachable objs:  55
Remaining Garbage:
[]

unreachable objs:  0
Remaining Garbage:
[]


In [2]:
import gc
import pprint

class Graph:
    def __init__(self, name):
        self.name = name
        self.next = None
        
    def set_next(self, next):
        print(f'link nodes {self}.next={next}')
        self.next = next
        
    def __repr__(self):
        return f"{self.__class__.__name__}({self.name})"
    
    def __del__(self):
        # 在jupyter中未被调用，但在gc_get_referrers.py中是可以的
        print(f"{self}.__del__()")
    
one = Graph('one')
two = Graph('two')
three = Graph('three')
one.set_next(two)
two.set_next(three)
three.set_next(one)

print('Collecting...')
n = gc.collect()
print('unreachable objs: ',n)
print('Remaining Garbage:')
pprint.pprint(gc.garbage)

# 定义我们要从本模块的本地变量，全局变量和垃圾回收器自己的记录中忽略一些引用
REFERERS_TO_IGNORE = [locals(), globals(), gc.garbage]

def find_referring_graphs(obj):
    print('looking for reference to ',obj)
    referers = (r for r in gc.get_referrers(obj) if r not in REFERERS_TO_IGNORE)
    for ref in referers:
        if isinstance(ref, Graph):
            yield ref
        elif isinstance(ref, dict):
            for parent in find_referring_graphs(ref):
                yield parent
                
print()
print('clearing referrers:')
for obj in [one, two, three]:
    for ref in find_referring_graphs(obj):
        print('found referrer:', ref)
        ref.set_next(None)
        del ref
    del obj

print()
print('clearing gc.garbage:')
del gc.garbage[:]

print('Collecting...')
n = gc.collect()
print('unreachable objs: ',n)
print('Remaining Garbage:')
pprint.pprint(gc.garbage)

Graph(one).__del__()
Graph(two).__del__()
Graph(three).__del__()
link nodes Graph(one).next=Graph(two)
link nodes Graph(two).next=Graph(three)
link nodes Graph(three).next=Graph(one)
Collecting...
unreachable objs:  9
Remaining Garbage:
[]

clearing referrers:
looking for reference to  Graph(one)
looking for reference to  {'name': 'three', 'next': Graph(one)}
found referrer: Graph(three)
link nodes Graph(three).next=None
looking for reference to  Graph(two)
looking for reference to  {'name': 'one', 'next': Graph(two)}
found referrer: Graph(one)
link nodes Graph(one).next=None
looking for reference to  Graph(three)
looking for reference to  {'name': 'two', 'next': Graph(three)}
found referrer: Graph(two)
link nodes Graph(two).next=None

clearing gc.garbage:
Collecting...
unreachable objs:  0
Remaining Garbage:
[]


## 回收阈值和代数

In [3]:
gc.get_threshold()

(700, 10, 10)

In [4]:
import sys

threshold = 5

class MyObj:
    def __init__(self, name):
        self.name = name
        print('created',name)
gc.set_debug(gc.DEBUG_STATS)
gc.set_threshold(threshold, 1, 1)
print('Thresholds:', gc.get_threshold())

print('clear the collector by forcing a run')
gc.collect()

print('creating objects')
objs = []
for i in range(10):
    objs.append(MyObj(i))
    
print('exiting')
gc.set_debug(0)

Thresholds: (5, 1, 1)
clear the collector by forcing a run
creating objects
created 0
created 1
created 2
created 3
created 4
created 5
created 6
created 7
created 8
created 9
exiting


gc: collecting generation 0...
gc: objects in each generation: 534 0 52088
gc: done, 0.0001s elapsed
gc: collecting generation 0...
gc: objects in each generation: 26 509 52088
gc: done, 0.0000s elapsed
gc: collecting generation 1...
gc: objects in each generation: 17 551 52088
gc: done, 0.0001s elapsed
gc: collecting generation 0...
gc: objects in each generation: 37 0 52666
gc: done, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 30 47 52651
gc: done, 0.0000s elapsed
gc: collecting generation 1...
gc: objects in each generation: 17 90 52651
gc: done, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 17 0 52774
gc: done, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 17 35 52772
gc: done, 0.0000s elapsed
gc: collecting generation 1...
gc: objects in each generation: 17 68 52772
gc: done, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 35 0 52858
gc: done, 0.0000s